A common problem in high-availability systems is how much resources to provide: how to ensure stability reducing the spending? A solution is auto-scaling: a system component that increases or decreases de amount of resources used by the system according to the current needs. But for that, we need to have an idea of how many requests will arrive. This is an interesting problem to model statistically.

Consider an employee portal. It is almost never accessed outside the work hours. During working hours , it usually has a predictably mild number of access during most of the workdays of the month. In the last working day of month, however, it have a spike of access from employees looking for correcting clock-in registrations that may be missing. Then, on the latest working day before or equal to 5, there is another spike, since employees receive their payment this day and want to check their payslip.

If we consider access in general as defined by a Poisson process, we can consider choosing a nonhomogeneous Poisson distribution.

Let's consider a scenario where we have 1 request per five minutes outside office hours. IN office hours, we have around 100 request per minute, except on the last day of the month, when it goes to 800 requests per minute, and on day 5, when it goes to 500 requests per minute. In this case, the rate function $\lambda$ responds for the day of the month:

In [2]:
import pandas as pd
from brandedpipes import __

from datetime import datetime, timedelta
from time import strptime

def rate_function(
        date,
        outside_office_hours_rate=0.2,
        office_hours_rate=100,
        payday_rate=500,
        clockin_rate=800,
    ):
    """
    Return the rate of access to the web app per minute for every day.

    It is very low outside business days...

    >>> rate_function(D('2024-05-11 9:40'))
    0.2

    ...and business hours:

    >>> rate_function(D('2024-12-19 7:00'))
    0.2

    Usually it is quite mild during office hours:

    >>> rate_function(D('2024-12-19 9:40'))
    100

    Every payday (day 5 or earliest), though, it spikes to 500 req/minute:

    >>> rate_function(D('2024-11-05 12:55'))
    500
    >>> rate_function(D('2024-10-04 12:59'))
    500

    In the last workday of the month, it spikes to 800 reqs/minunte due to
    clock-in adjustments

    >>> rate_function(D('2024-02-29 10:00'))
    800
    >>> rate_function(D('2024-6-28 10:00'))
    800
    """
    if not is_office_hours(date):
        return outside_office_hours_rate
    if is_last_business_day_wrt(date, 5):
        return payday_rate
    if is_last_business_day_wrt(date, last_day_of_month(date)):
        return clockin_rate
    return office_hours_rate

def is_office_hours(
        date,
        office_hours_start=8,
        office_hours_end=18
    ):
    if not is_business_day(date):
        return False
    if date.hour < office_hours_start:
        return False
    if date.hour > office_hours_end-1:
        return False
    return True

def last_day_of_month(date):
    """
    >>> last_day_of_month(D('2012-02-14'))
    29
    """
    first_day_current_month = datetime(date.year, date.month, 1)
    next_month = first_day_current_month + timedelta(days=31)
    first_day_next_month = datetime(next_month.year, next_month.month, 1)
    return previous_day(first_day_next_month).day

def is_last_business_day_wrt(date, day):
    """
    Receives a date and a day of the month. Returns true if
    the date is the day and is also a business day:

    >>> is_last_business_day_wrt(D('2025-09-05'), 5)
    True

    If the date is not in that day of the month, and in that
    month the given day is not a business day, it returns
    True if the given date is the last business day before
    the given day:

    >>> is_last_business_day_wrt(D('2024-10-04'), 5)
    True
    >>> is_last_business_day_wrt(D('2025-09-5'), 7)
    True

    It will return false if the date is not a business day...

    >>> is_last_business_day_wrt(D('2024-10-5'), 5)
    False

    ...or if it is not the given day of the month:

    >>> is_last_business_day_wrt(D('2024-10-3'), 4)
    False
    """
    if not is_business_day(date):
        return False
    if date.day == day:
        return True
    nd = next_day(date)
    while not is_business_day(nd):
        if nd.day == day:
            return True
        nd = next_day(nd)
    return False

def is_business_day(date):
    """
    Returns true if the date is a business date (not considering holidays):

    >>> is_business_day(D('2025-09-05'))
    True
    >>> is_business_day(D('2025-09-06'))
    False
    """
    return __| pd.bdate_range(start=date, end=date) | len | bool |__

def previous_day(date):
    return date + timedelta(days=-1)

def next_day(date):
    return date + timedelta(days=1)

def D(s):
    try:
        sd = strptime(s, '%Y-%m-%d %H:%M')
    except ValueError as e:
        sd = strptime(s, '%Y-%m-%d')
    return datetime(*sd[:5])